<center><h1>Стекинг моделей.</h1></center>

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import (
    f1_score, 
    accuracy_score,
    classification_report, 
)

# <center> 1. Подготовка обучающих данных </center>

## 1.1. Получение данных

In [2]:
train_data = pd.read_csv("../../data/original/train.csv.csv", index_col=0)
assessment = train_data['assessment']

text = pd.read_csv("../../data/processed/text_df.csv", index_col=0)
tags = pd.read_csv("../../data/processed/tags_df.csv", index_col=0)
targets = pd.read_csv("../../data/processed/target_df.csv", index_col=0)

In [3]:
text = text.fillna("")

## 1.2. Получение эмбеддингов текстов

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=8000)
train_embeddings = tfidf.fit_transform(text['text']).toarray()

## 1.3. Формирование итоговых датасетов для обучения 

In [6]:
final_train_data_1 = pd.concat([pd.DataFrame(assessment.values), pd.DataFrame(train_embeddings)], axis=1).values
final_train_data_2 = pd.concat([pd.DataFrame(assessment.values), pd.DataFrame(tags.values), pd.DataFrame(train_embeddings)], axis=1).values

# <center> 2. Обучение моделей </center>

In [10]:
from sklearn.multioutput import MultiOutputClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import StackingClassifier

In [11]:
estimators = [('logreg', MultiOutputClassifier(LogisticRegression(random_state=42))),
              ('tree_1', MultiOutputClassifier(DecisionTreeClassifier(max_depth=33, min_samples_leaf=6, random_state=42))),
              ('forest', MultiOutputClassifier(RandomForestClassifier(random_state=42))),
              ('naive', MultiOutputClassifier(ComplementNB()))]

final_estimator = MultiOutputClassifier(GradientBoostingClassifier(n_estimators=20, random_state=42))

In [12]:
model = StackingClassifier(estimators=estimators, final_estimator=final_estimator)
model.fit(final_train_data_1, targets.values)

StackingClassifier(estimators=[('logreg',
                                MultiOutputClassifier(estimator=LogisticRegression(random_state=42))),
                               ('tree_1',
                                MultiOutputClassifier(estimator=DecisionTreeClassifier(max_depth=33,
                                                                                       min_samples_leaf=6,
                                                                                       random_state=42))),
                               ('forest',
                                MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))),
                               ('naive',
                                MultiOutputClassifier(estimator=ComplementNB()))],
                   final_estimator=MultiOutputClassifier(estimator=GradientBoostingClassifier(n_estimators=20,
                                                                                              random_state=42)))

# <center> 3. Подготовка тестовых данных </center> 

## 3.1. Получение данных

In [13]:
test_data = pd.read_csv("../../data/original/test.csv.csv", index_col=0)
test_assessment = test_data['assessment']

test_data.head()

,index,assessment,tags,text
1,3135,3.0,{DELIVERY},"Последнее время думаю плохо, сроки доставки да..."
3,4655,2.0,"{PRICE,DELIVERY,ASSORTMENT}",Цены намного выше магазинных но радуют акции
5,22118,2.0,"{CATALOG_NAVIGATION,ASSORTMENT,DELIVERY}","Доставка за [NUM] минут, заказ даже не начали ..."
7,23511,0.0,{DELIVERY},Ужасно долгая доставка
8,45,6.0,"{ASSORTMENT,PROMOTIONS}",Добрый вечер! Вы большие молодцы. Меня всё уст...


## 3.2. Предобработка текстовых данных

In [14]:
import re
from bs4 import BeautifulSoup


def preprocessor(text):
    
    text = BeautifulSoup(text, "html.parser").get_text() # Удаляем HTML
    text = re.sub(r'http\S+', '', text)  # Удаление ссылок
    text = re.sub(r'\d+', '', text)  # Удаление номеров
    text = re.sub(r'\s+', ' ', text) # удаление лишних пробелов
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9\.,?!\s]', '', text) # Удаление всех символов кроме значимых(буквы, пунктуация)
    text = text.lower()
    
    return text

In [15]:
test_text = pd.DataFrame(test_data['text'].values, columns=['text'])
test_text = test_text.fillna("")
test_text['text'] = test_text['text'].apply(preprocessor)
test_text.head()

C:\Users\Максат\AppData\Local\Temp\ipykernel_9400\731392797.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text() # Удаляем HTML


,text
0,"последнее время думаю плохо, сроки доставки да..."
1,цены намного выше магазинных но радуют акции
2,"доставка за num минут, заказ даже не начали со..."
3,ужасно долгая доставка
4,добрый вечер! вы большие молодцы. меня вс устр...


## 3.3. Получение эмбеддингов текстов

In [16]:
test_embeddings = tfidf.transform(test_text['text']).toarray()

## 3.4. Работа с признаком тегов

In [17]:
test_data[test_data['tags'].isna()] = test_data[test_data['tags'].isna()].fillna('')

tags_set = {'ASSORTMENT',
 'CATALOG_NAVIGATION',
 'DELIVERY',
 'PAYMENT',
 'PRICE',
 'PRODUCTS_QUALITY',
 'PROMOTIONS',
 'SUPPORT'}

def string_to_set(string):
  elements = string.strip('{}').split(',')
  set_result = set(element.strip() for element in elements)
  return set_result


def make_tags_df(df, tags_set):
    tags_list = list(tags_set)
    tags_list = sorted(tags_list)
    new_df = pd.DataFrame(columns=tags_list)

    for _, row in df.iterrows():
        tags = string_to_set(row['tags'])
        for tag in tags_list:
            if tag in tags:
                new_df.loc[row.name, tag] = 1
            else:
                new_df.loc[row.name, tag] = 0
    return new_df


test_tags = pd.DataFrame(make_tags_df(test_data, tags_set).values, columns=sorted(list(tags_set)))
test_tags.head()

,ASSORTMENT,CATALOG_NAVIGATION,DELIVERY,PAYMENT,PRICE,PRODUCTS_QUALITY,PROMOTIONS,SUPPORT
0,0,0,1,0,0,0,0,0
1,1,0,1,0,1,0,0,0
2,1,1,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,1,0


## 3.5. Итоговые тестовые данные и предсказания 

In [18]:
final_test_data_1 = pd.concat([pd.DataFrame(test_assessment.values), pd.DataFrame(test_embeddings)], axis=1).values
final_test_data_2 = pd.concat([pd.DataFrame(test_assessment.values), pd.DataFrame(test_tags.values), pd.DataFrame(test_embeddings)], axis=1).values

In [19]:
prediction = model.predict(final_test_data_1)

In [21]:
def get_class_labels(array):
    
  class_labels = []
  for i in range(array.shape[0]):
    classes = np.where(array[i] == 1)[0]
    if len(classes) > 0:
      class_labels.append(" ".join(str(x) for x in classes))
    else:
      class_labels.append("")


  df = pd.DataFrame({"class_labels": class_labels})

  return df

answer_1 = get_class_labels(prediction)

In [22]:
answer_1 = pd.DataFrame(answer_1.values, index=test_data['index'].values, columns=['target'])
answer_1.to_csv('../../results/submissions/4-1.csv')